In [20]:
#importing libs
import pandas as pd

In [41]:
#reading the dataset
data_train = pd.read_csv('../dataset/train_data_covid.csv')
data_test = pd.read_csv('../dataset/test_data_covid.csv')


In [49]:
data_train

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1,8262,867
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1,9136,61
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2,3826,316
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3,9789,806
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3,882,369
...,...,...,...,...,...,...,...,...,...,...,...
14996,14997,2021-05-17,8:00 AM,Lakshadweep,-,-,3633,14,4767,4046,490
14997,14998,2021-05-17,8:00 AM,Madhya Pradesh,-,-,629741,6992,731385,1195,595
14998,14999,2021-05-17,8:00 AM,Maharashtra,-,-,4826371,81486,5378452,9289,195
14999,15000,2021-05-17,8:00 AM,Manipur,-,-,32674,578,39729,576,296


In [42]:
data_test

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,15001,2021-05-17,8:00 AM,Meghalaya,-,-,18478,23332,191,514
1,15002,2021-05-17,8:00 AM,Mizoram,-,-,6687,8829,1111,563
2,15003,2021-05-17,8:00 AM,Nagaland,-,-,13767,18076,9896,577
3,15004,2021-05-17,8:00 AM,Odisha,-,-,514532,612224,6091,615
4,15005,2021-05-17,8:00 AM,Puducherry,-,-,65689,84506,220,573
...,...,...,...,...,...,...,...,...,...,...
3105,18106,2021-08-11,8:00 AM,Telangana,-,-,638410,650353,8796,659
3106,18107,2021-08-11,8:00 AM,Tripura,-,-,77811,80660,9330,488
3107,18108,2021-08-11,8:00 AM,Uttarakhand,-,-,334650,342462,6290,686
3108,18109,2021-08-11,8:00 AM,Uttar Pradesh,-,-,1685492,1708812,723,115


In [45]:
#taking the required features
data_train_abs = data_train[["Sno", "Date", "State/UnionTerritory", "Cured", "Deaths", "Confirmed", "OxygenTanks", "PopulationDensityPerSqKm"]]

In [46]:
data_train_abs

,Sno,Date,State/UnionTerritory,Cured,Deaths,Confirmed,OxygenTanks,PopulationDensityPerSqKm
0,1,2020-01-30,Kerala,0,0,1,8262,867
1,2,2020-01-31,Kerala,0,0,1,9136,61
2,3,2020-02-01,Kerala,0,0,2,3826,316
3,4,2020-02-02,Kerala,0,0,3,9789,806
4,5,2020-02-03,Kerala,0,0,3,882,369
...,...,...,...,...,...,...,...,...
14996,14997,2021-05-17,Lakshadweep,3633,14,4767,4046,490
14997,14998,2021-05-17,Madhya Pradesh,629741,6992,731385,1195,595
14998,14999,2021-05-17,Maharashtra,4826371,81486,5378452,9289,195
14999,15000,2021-05-17,Manipur,32674,578,39729,576,296
